# CFM Challenge
2018-07. 

_Note_ : This model gives lower scores than the actual results on the challenge : 
   - Indeed the notebook will give you around ~21.35 by averaging the 5kfolds which will give around ~20.94 on the public leaderboard. This might be due to high regularisation on the model and also because the test set give better predictions than the train set. 

To get the best score  on the leaderboard at the end of the competition (around ~20.88), I've just averaged the results of the top 10 best csvs results with different models I've tried during the competion.
The different models tried were always pretty the sames. What basically changes is the size of the layers, the concatenation/multiplication/addition in the model, the way you deal with nans in the dataset and what kind of engineered features you will take in input + embeddings size (ok that's a lot of changes... but no matter what you do, you will have more or else the same result with this model).



If you want more details about the model and how does it work, you can find it on the attached slides. This presentation was given at CFM and the end of the academic competition and gives some insights of what worked and what did not.

PS : The script isn't really well coded...  basically because I didn't really take the time to do a whole refactoring and it was made of a lot of iterations. So some stuff should not be done like this anymore and doesn't really make sens.. (sorry for that !) 

If you have any question or suggestions (or if you get way better scores than me !), let me know, I would be happy to discuss and learn ;).


#### Let's import some stuff

In [1]:
import datetime
import numpy as np
import pandas as pd

from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.layers import multiply, Layer, LeakyReLU, PReLU, Bidirectional, merge, Dense, GlobalAveragePooling1D, SpatialDropout1D,concatenate,add,Embedding,Input,Masking,LSTM,Flatten,TimeDistributed,GlobalMaxPooling1D,GRU,Activation,Dropout,Reshape, BatchNormalization
from sklearn.preprocessing import LabelEncoder, Imputer, StandardScaler, RobustScaler, MinMaxScaler
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau,TensorBoard, LearningRateScheduler
from keras import losses
from keras import initializers, regularizers, constraints
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold,train_test_split
import keras
import hashlib
import math


#These libs are not from me, I've just copy past the code to use them... 
#You can find the original repos below and the papers associated in the code.
from CLR import CyclicLR #https://github.com/bckenstler/CLR/blob/master/clr_callback.py | https://arxiv.org/abs/1506.01186
from janet import JANET #https://github.com/titu1994/Keras-just-another-network-JANET/blob/master/janet.py | https://arxiv.org/abs/1804.04849
import gc
print(keras.__version__)


/Users/cdelabre/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


2.1.3


#### Get the data

In [2]:
home_dir = "data/"
def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

id_col = "ID"
target_col = "TARGET"


In [3]:
train_df = pd.read_csv("%s/training_input.csv" % home_dir, sep=";")
id_df = pd.read_csv("%s/id.csv" % home_dir, sep=";")
test_df = pd.read_csv("%s/testing_input.csv" % home_dir, sep=";")
train_df  = train_df.merge(id_df, on="ID")
del id_df; gc.collect()

21

In [4]:
volatility_cols = [c for c in train_df.columns if c.startswith("volatility")]
return_cols = [c for c in train_df.columns if c.startswith("return")]
other_cols = ["date" , "product_id"]

#### Some feature engineering

In [5]:
#sum of nans
for df in [train_df,test_df]:
    df["vol_nan"] = df[volatility_cols].isnull().sum(axis=1)

In [6]:
#replace nans by interpolation
for df in [train_df,test_df]:
    for x in [volatility_cols,return_cols]:
        df[x] = df[x].interpolate(axis=1, limit_direction="both", inplace=False)

In [7]:
def get_mean_groupby(all_data, groupby_cols):
    for groupby_obj in groupby_cols:
        groupby_col = groupby_obj["id"]
        print(groupby_col)
        cols = groupby_obj["cols"]
        group_by = all_data.groupby([groupby_col])
        data_arr = []
        data_arr.append({"i": "avg", "d": group_by[cols].mean()})
        data_arr.append({"i": "skew", "d": group_by[cols].skew()})
        data_arr.append({"i": "kurt", "d": group_by[cols].apply(pd.DataFrame.kurt)})
        data_arr.append({"i": "std", "d": group_by[cols].std()})
        data_arr.append({"i": "median", "d": group_by[cols].median()})
        data_arr.append({"i": "nan", "d": all_data.isnull().groupby(all_data[groupby_col])[cols].sum()})

        all_data.set_index([groupby_col], inplace=True)
        for obj_data in data_arr:
            names = ['%s_%s_%s' % (obj_data["i"], groupby_col, col) for col in cols]            
            all_data[names] = (obj_data["d"]).astype("float32")
        all_data.reset_index(inplace=True)
    return all_data

def group_by_date_countd(all_data):
    groupby_col = "date"
    unique_products = all_data.groupby([groupby_col])["product_id"].nunique()
    all_data.set_index([groupby_col], inplace=True)
    all_data["countd_product"] = unique_products.astype('uint16')
    all_data.reset_index(inplace=True)
    return all_data


def group_by_product_countd(all_data):
    groupby_col = "product_id"
    unique_date = all_data.groupby([groupby_col])["date"].nunique()

    all_data.set_index([groupby_col], inplace=True)
    all_data["countd_date"] = unique_date.astype('uint16')
    all_data.reset_index(inplace=True)
    return all_data

In [8]:
#group by date and product_id to get more features
calculation_group_by =[
    {"id":"date" ,
     "cols": volatility_cols + return_cols,
    },
    {"id":"product_id" ,
     "cols": volatility_cols + return_cols,
    }
]
train_df["is_train"] = True
test_df["is_train"] = False
test_df["TARGET"] = None
all_data = pd.concat([train_df, test_df])
del train_df
del test_df

print("Creating new aggregated features")
all_data = get_mean_groupby(all_data, calculation_group_by)
print("Group by date")
all_data = group_by_date_countd(all_data)
print("Group by product")
all_data = group_by_product_countd(all_data)

Creating new aggregated features
date
product_id
Group by date
Group by product


In [9]:
#replace nan per 0
rep_values= 0
all_data = all_data.replace([np.inf, -np.inf], rep_values)
all_data.fillna(rep_values, inplace=True)

## Prepare to Train

In [10]:
#get all cols per type
#TODO refactor to do it in a for loop 
volatility_avg_date_cols = ["avg_date_%s" % x for x in volatility_cols]
volatility_avg_product_cols = ["avg_product_id_%s" % x for x in volatility_cols]
return_avg_date_cols = ["avg_date_%s" % x for x in return_cols]
return_avg_product_cols = ["avg_product_id_%s" % x for x in return_cols]

volatility_std_date_cols = ["std_date_%s" % x for x in volatility_cols]
volatility_std_product_cols = ["std_product_id_%s" % x for x in volatility_cols]
return_std_date_cols = ["std_date_%s" % x for x in return_cols]
return_std_product_cols = ["std_product_id_%s" % x for x in return_cols]

volatility_median_date_cols = ["median_date_%s" % x for x in volatility_cols]
volatility_median_product_cols = ["median_product_id_%s" % x for x in volatility_cols]

volatility_kurt_date_cols = ["kurt_date_%s" % x for x in volatility_cols]
volatility_kurt_product_cols = ["kurt_product_id_%s" % x for x in volatility_cols]
return_kurt_date_cols = ["kurt_date_%s" % x for x in return_cols]
return_kurt_product_cols = ["kurt_product_id_%s" % x for x in return_cols]

volatility_nan_date_cols = ["nan_date_%s" % x for x in volatility_cols]
volatility_nan_product_cols = ["nan_product_id_%s" % x for x in volatility_cols]

In [11]:
#encode and preprocess data

def fit_encoder(obj_in_cols_obj, all_data, encoder_name="StandardScaler"):
    if encoder_name is not None:
        for i,x in enumerate(obj_in_cols_obj):
            group_cols = x["cols"] #group of lstm cols
            encoders = []
            for cols in group_cols:
                X_all = all_data[cols].values
                encoder = df_mapper(cols, X_all, encoder_name)
                encoders.append(encoder)
            obj_in_cols_obj[i]["encoders"] = encoders
    return obj_in_cols_obj

def df_mapper(cols, all_data , encoder):
    if encoder == "StandardScaler":
        encoder = StandardScaler() 
    elif encoder == "LabelEncoder":
        encoder = LabelEncoder()
    elif encoder == "RobustScaler":
        encoder = RobustScaler()
    elif encoder == "MinMaxScaler":
        encoder = MinMaxScaler(feature_range=(0,1))
    else:
        raise NotFoundException()
        
    encoder = encoder.fit(all_data)
    return encoder
 
def prepare_data(lstm_in_cols_obj, emb_in_cols_obj, df_to_prepare):
    def __prepare_data_inn(cols, df_tmp,  encoders, encode=True, reshape=True):
        Xs = []
        for i, col in enumerate(cols):
            X = df_tmp[col].values # timeseries cols
            if encode and encoders[i] is not None:
                X = encoders[i].transform(X)
            if reshape:
                X = X.reshape(X.shape[0], X.shape[1], 1)
            Xs.append(X)
        X = np.concatenate([x for x in Xs] , axis=len(Xs[0].shape)-1) if len(Xs) > 1 else Xs[0] 
        return X
       
    Xs_map = []
    for x in lstm_in_cols_obj:
        cols = x["cols"]
        encoders = x["encoders"] if "encoders" in x else None
        encode = True if "encoders" in x else False
        Xs_map.append(__prepare_data_inn(cols, df_tmp=df_to_prepare, reshape=True, encoders=encoders, encode=encode))
    #TODO Refactor ravel
    embs = []
    for x in emb_in_cols_obj:
        cols = x["cols"]
        encoders = x["encoders"] if "encoders" in x else None
        encode = True if "encoders" in x else False
        X_emb = __prepare_data_inn(cols, df_tmp=df_to_prepare, reshape=False, encoders=encoders, encode=encode)
        embs.append(X_emb)
    X_t = [x for x in Xs_map] + embs

    return X_t


def generate(df_generator, target, batch_size, lstm_in_cols_obj,emb_in_cols_obj, y_vals=True, shuffle=True):
    imax = int(math.ceil((df_generator.shape[0])/batch_size))
    while True:
        indexes = __get_exploration_order(df_generator, shuffle)
        for i in range(imax):
            upp = min((i+1)*batch_size, df_generator.shape[0])
            df_tmp = df_generator.loc[indexes[i*batch_size:upp]]
            X = prepare_data(lstm_in_cols_obj,emb_in_cols_obj, df_tmp)
            if y_vals:
                y = df_tmp[target].values
                yield X, y
            else:
                yield X

                
def __get_exploration_order(df, shuffle=True):
    indexes = df.index.values
    if shuffle == True:
        np.random.shuffle(indexes)
    return indexes


In [12]:
#create the model
def create_model(lstm_in_cols_obj, emb_in_cols_obj):
    def create_emb(name, out_len, in_len):
        inp = Input(shape=[1], dtype='int64', name="%s_in" % name)
        out = (Embedding(in_len, out_len, name="%s_emb" % name)(inp))
        out = SpatialDropout1D(0.2)(out)
        out = Flatten()(out)
        return inp,out


    def create_lstm(name, lstm_out, timesteps_in, dims_in):
        model_inp = Input(shape=(timesteps_in,dims_in), name="%s_in" % name)
        model_out = JANET(lstm_out,return_sequences=False,name="%s_lstm" % name)(model_inp)
        return model_inp, model_out

    lstm_in_array= []
    lstm_out_array = []
    for x in lstm_in_cols_obj:
        name = x["name"]
        dims_in = len(x["cols"])
        lstm_out = x["out_l"]
        x_model_in, x_model_out = create_lstm(name=name, lstm_out=lstm_out, timesteps_in=54, dims_in=dims_in)
        lstm_in_array.append(x_model_in)
        lstm_out_array.append(x_model_out)
    
    #add first two
    model_lstm_add_two = add([lstm_out_array[0] , lstm_out_array[1]])
    model_lstm_add_two = PReLU()(model_lstm_add_two)
    #add last three lstm
    model_lstm_add_three = add([lstm_out_array[2] , lstm_out_array[3] , lstm_out_array[4]])
    model_lstm_add_three = PReLU()(model_lstm_add_three)

    #concat with add layers + origin layer
    model_lstm = concatenate([model_lstm_add_three] + [model_lstm_add_two] + lstm_out_array)
    model_lstm = (Dense(200)(model_lstm))
    model_lstm = PReLU()(model_lstm)

    #create embeddings
    in_array_emb= []
    out_array_emb = []
    for x in emb_in_cols_obj:  
        in_len = x["in_l"]
        out_len = x["out_l"]
        name = x["name"]
        inp_emb, out_emb = create_emb(name=name , out_len=out_len, in_len = in_len)
        in_array_emb.append(inp_emb)
        out_array_emb.append((out_emb))
        
    model_emb = concatenate(out_array_emb) if len(out_array_emb) > 1 else out_array_emb[0]
    model_emb = Dropout(0.2)(Dense(100)(model_emb))
    model_emb = PReLU()(model_emb)
    
    #concatenate embeddings & lstms
    model = concatenate([model_emb] + [model_lstm]+ [model_lstm_add_two])

    model = Dense(300)(model)
    model = PReLU()(model)
    model = Dropout(0.2)(model)
    model = Dense(1, activation="linear")(model)
    model =  Model(lstm_in_array+in_array_emb, model)
    
    return model
    

Params for model

In [13]:
#params of the model
lstm_in_cols_obj = [
{"name" : "volatility" , "cols": [volatility_cols] , "out_l": 45} ,
{"name" : "return" , "cols": [return_cols] , "out_l": 45} ,
{"name" : "volatility_per_product" , "cols": [volatility_avg_product_cols, volatility_median_product_cols, volatility_std_product_cols, volatility_nan_product_cols], "out_l": 15} ,
{"name" : "volatility_per_date" , "cols": [volatility_avg_date_cols, volatility_median_date_cols, volatility_std_date_cols, volatility_nan_date_cols], "out_l": 15} , 
{"name" : "return_per_date_and_product" , "cols": [return_avg_date_cols, return_kurt_date_cols , return_avg_product_cols, return_kurt_product_cols ], "out_l": 15}
]

emb_in_cols_obj = [
    {"name" : "emb_product_id" , "cols": [["product_id"]] , "out_l": 80, "in_l" : all_data["product_id"].nunique()} ,
    {"name" : "countd_date" , "cols": [["countd_date"]] , "out_l": 10 , "in_l" : all_data["countd_date"].nunique()} ,
    {"name" : "countd_product" , "cols": [["countd_product"]] , "out_l": 5 , "in_l" : all_data["countd_product"].nunique()} ,
    {"name" : "countd_vol_nan" , "cols": [["vol_nan"]] , "out_l": 5 , "in_l" : all_data["vol_nan"].nunique()}
]

In [14]:
### add encoders in lstm_in_cols_obj 
lstm_in_cols_obj = fit_encoder(lstm_in_cols_obj, all_data , encoder_name="StandardScaler")
emb_in_cols_obj = fit_encoder(emb_in_cols_obj, all_data , encoder_name="LabelEncoder")

/Users/cdelabre/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
test_df = all_data[all_data["is_train"] == False]
train_df = all_data[all_data["is_train"] == True]
del all_data;

In [16]:
X_test = prepare_data(lstm_in_cols_obj, emb_in_cols_obj, test_df)

/Users/cdelabre/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [17]:
#model = create_model(lstm_in_cols_obj, emb_in_cols_obj)
#model.summary()

### Train the model with 5 KFold 

In [18]:
#sort values in another order
#sort values per date in order to NOT train with dates (validation is done with dates never seen like in the test set) 
seed = 42
train_df["shuffle"] = train_df['date'].apply(lambda x: hashlib.sha256(str(x*seed)).hexdigest())
train_df.sort_values(["shuffle"], inplace=True)
monitor = "val_loss"
model_prefix = "cfm-example-model-%s" % datetime.datetime.now().strftime("%Y-%m-%d")
total_folds = 5
total_it = 120 

In [19]:
kfold = KFold(n_splits=total_folds, shuffle=False)
cvscores = []
test_preds = []
batch_size = (len(train_df) - len(train_df)//total_folds)//(total_it)
i = 1
for train, valid in kfold.split(train_df):
    tmp_df_train = train_df.loc[train].reset_index(drop=True)
    tmp_df_valid = train_df.loc[valid].sample(frac=1).reset_index(drop=True)
    print("Train %d , Valid %d" % (len(tmp_df_train) , len(tmp_df_valid)))

    base_lr = 0.001
    j = 1 
    last_loss_score = 100 #default init.
    model = create_model(lstm_in_cols_obj, emb_in_cols_obj)
    once = False
    print("Start Fold %i - start base_lr %0.4f : " % (i , base_lr))
    while True:
        model_checkpoint_name_tmp = "weights/%s-best-fold-%d-it-%d.h5" % (model_prefix, i, j )
        callbacks = [EarlyStopping(monitor=monitor, patience=3, verbose=1, min_delta=1e-8, mode='min'),
                     ModelCheckpoint(model_checkpoint_name_tmp, monitor=monitor, save_best_only=True, save_weights_only=True, verbose=1, mode="min")
                    ]
        step_size = 100
        clr  = CyclicLR(mode='exp_range', max_lr=base_lr*6, base_lr=base_lr, step_size=step_size, logs=False)
        callbacks.append(clr)
        
        train_gen = generate(tmp_df_train, target_col, batch_size, lstm_in_cols_obj, emb_in_cols_obj, True, True)
        valid_gen = generate(tmp_df_valid, target_col, batch_size, lstm_in_cols_obj, emb_in_cols_obj, True, False)
        
        model.compile(loss= "mean_absolute_percentage_error",                      
                  optimizer=Adam(),
                      metrics=["mae"])
        hist = model.fit_generator(
            generator=train_gen,
            validation_data=valid_gen,
            steps_per_epoch = math.ceil((tmp_df_train.shape[0])/batch_size),
            validation_steps = math.ceil((tmp_df_valid.shape[0])/batch_size),
            verbose=1,epochs=50,
            callbacks = callbacks,
            max_queue_size=30, workers=8
                 )         
        model.load_weights(model_checkpoint_name_tmp)
        loss_score,score = model.evaluate_generator(valid_gen, steps=math.ceil((tmp_df_valid.shape[0])/batch_size))
        print("Checkpoint. Fold %i | Iteration %i (LR %0.6f) -- Score --> %0.8f | %0.8f "  % (i, j , base_lr , score,  loss_score))
        if loss_score > last_loss_score:
            print("Score Worse than before (%0.8f | %0.8f)" % ( loss_score , last_loss_score))
            if once:
                break
            once = True
            print("Try one last time reducing LR.")
            base_lr = base_lr/10

        else:
            once = False
            print("Score Better than before (%0.8f | %0.8f)" % (loss_score ,last_loss_score))
            last_loss_score=loss_score
            base_lr = base_lr/5
            best_j = j 
            print("Reducing LR to %f" % base_lr)
        if j >3:
            print("Stop. Too Much Iterations.")
            break
        model_checkpoint_name_tmp = "weights/%s-best-fold-%d-it-%d.h5" % (model_prefix, i, best_j )
        model.load_weights(model_checkpoint_name_tmp)
        j = j+1
            

    print("End of Training for Fold %i. Predicting."  % (i))
    #GET LAST SCORE
    model_checkpoint_name = "weights/%s-best-fold-%d-it-%d.h5" % (model_prefix, i, best_j )
    model.load_weights(model_checkpoint_name)
    #save best model
    model.save_weights("weights/%s-best-fold-%d-Best.h5" % (model_prefix, i) , overwrite=True )
    score,score_mae = model.evaluate_generator(valid_gen, steps=math.ceil(len(tmp_df_valid)/batch_size))

    test_yPreds = model.predict(X_test, batch_size=2048, verbose=0)
    cvscores.append((score_mae,score))
    test_preds.append(test_yPreds[:,-1])
    
    print("End of Fold %i | Score --> %0.4f | %0.4f"  % (i, score, score_mae) )
    i=i+1

Train 509050 , Valid 127263
Start Fold 1 - start base_lr 0.0010 : 
Epoch 1/50
119/120 [============================>.] - ETA: 1s - loss: 31.1951 - mean_absolute_error: 0.0545
Epoch 00001: val_loss improved from inf to 24.00898, saving model to weights/cfm-example-model-2018-07-05-best-fold-1-it-1.h5
120/120 [==============================] - 173s 1s/step - loss: 31.1370 - mean_absolute_error: 0.0545 - val_loss: 24.0090 - val_mean_absolute_error: 0.0451
Epoch 2/50
119/120 [============================>.] - ETA: 1s - loss: 22.2450 - mean_absolute_error: 0.0426
Epoch 00002: val_loss improved from 24.00898 to 22.33117, saving model to weights/cfm-example-model-2018-07-05-best-fold-1-it-1.h5
120/120 [==============================] - 162s 1s/step - loss: 22.2435 - mean_absolute_error: 0.0426 - val_loss: 22.3312 - val_mean_absolute_error: 0.0434
Epoch 3/50
119/120 [============================>.] - ETA: 1s - loss: 21.7590 - mean_absolute_error: 0.0413
Epoch 00003: val_loss improved from 22.3

Epoch 00007: early stopping
Checkpoint. Fold 1 | Iteration 3 (LR 0.000020) -- Score --> 0.04151054 | 21.57486013 
Score Better than before (21.57486013 | 21.62186356)
Reducing LR to 0.000004
Epoch 1/50
119/120 [============================>.] - ETA: 1s - loss: 20.8356 - mean_absolute_error: 0.0392
Epoch 00001: val_loss improved from inf to 21.59430, saving model to weights/cfm-example-model-2018-07-05-best-fold-1-it-4.h5
120/120 [==============================] - 255s 2s/step - loss: 20.8330 - mean_absolute_error: 0.0392 - val_loss: 21.5943 - val_mean_absolute_error: 0.0417
Epoch 2/50
119/120 [============================>.] - ETA: 1s - loss: 20.8432 - mean_absolute_error: 0.0392
Epoch 00002: val_loss improved from 21.59430 to 21.59142, saving model to weights/cfm-example-model-2018-07-05-best-fold-1-it-4.h5
120/120 [==============================] - 250s 2s/step - loss: 20.8442 - mean_absolute_error: 0.0392 - val_loss: 21.5914 - val_mean_absolute_error: 0.0416
Epoch 3/50
119/120 [====

119/120 [============================>.] - ETA: 1s - loss: 20.6994 - mean_absolute_error: 0.0389
Epoch 00002: val_loss improved from 21.30473 to 21.27991, saving model to weights/cfm-example-model-2018-07-05-best-fold-2-it-3.h5
120/120 [==============================] - 250s 2s/step - loss: 20.6971 - mean_absolute_error: 0.0389 - val_loss: 21.2799 - val_mean_absolute_error: 0.0377
Epoch 3/50
119/120 [============================>.] - ETA: 1s - loss: 20.6622 - mean_absolute_error: 0.0388
Epoch 00003: val_loss improved from 21.27991 to 21.27225, saving model to weights/cfm-example-model-2018-07-05-best-fold-2-it-3.h5
120/120 [==============================] - 249s 2s/step - loss: 20.6604 - mean_absolute_error: 0.0388 - val_loss: 21.2723 - val_mean_absolute_error: 0.0379
Epoch 4/50
119/120 [============================>.] - ETA: 1s - loss: 20.5907 - mean_absolute_error: 0.0387
Epoch 00004: val_loss improved from 21.27225 to 21.26652, saving model to weights/cfm-example-model-2018-07-05-be

119/120 [============================>.] - ETA: 1s - loss: 21.4926 - mean_absolute_error: 0.0405
Epoch 00005: val_loss did not improve
120/120 [==============================] - 254s 2s/step - loss: 21.4936 - mean_absolute_error: 0.0405 - val_loss: 21.2586 - val_mean_absolute_error: 0.0407
Epoch 6/50
119/120 [============================>.] - ETA: 1s - loss: 21.4877 - mean_absolute_error: 0.0403
Epoch 00006: val_loss did not improve
120/120 [==============================] - 258s 2s/step - loss: 21.4998 - mean_absolute_error: 0.0403 - val_loss: 22.9731 - val_mean_absolute_error: 0.0414
Epoch 7/50
119/120 [============================>.] - ETA: 1s - loss: 21.6222 - mean_absolute_error: 0.0403
Epoch 00007: val_loss did not improve
120/120 [==============================] - 254s 2s/step - loss: 21.6208 - mean_absolute_error: 0.0403 - val_loss: 21.2310 - val_mean_absolute_error: 0.0395
Epoch 00007: early stopping
Checkpoint. Fold 3 | Iteration 1 (LR 0.001000) -- Score --> 0.03998899 | 21.0

119/120 [============================>.] - ETA: 1s - loss: 22.3086 - mean_absolute_error: 0.0419
Epoch 00002: val_loss improved from 23.08213 to 22.03694, saving model to weights/cfm-example-model-2018-07-05-best-fold-4-it-1.h5
120/120 [==============================] - 248s 2s/step - loss: 22.3018 - mean_absolute_error: 0.0419 - val_loss: 22.0369 - val_mean_absolute_error: 0.0444
Epoch 3/50
119/120 [============================>.] - ETA: 1s - loss: 21.9641 - mean_absolute_error: 0.0409
Epoch 00003: val_loss did not improve
120/120 [==============================] - 250s 2s/step - loss: 21.9567 - mean_absolute_error: 0.0409 - val_loss: 22.3551 - val_mean_absolute_error: 0.0471
Epoch 4/50
119/120 [============================>.] - ETA: 1s - loss: 21.5951 - mean_absolute_error: 0.0401
Epoch 00004: val_loss did not improve
120/120 [==============================] - 250s 2s/step - loss: 21.5992 - mean_absolute_error: 0.0401 - val_loss: 22.3890 - val_mean_absolute_error: 0.0455
Epoch 5/50
1

End of Fold 4 | Score --> 21.5585 | 0.0429
Train 509051 , Valid 127262
Start Fold 5 - start base_lr 0.0010 : 
Epoch 1/50
119/120 [============================>.] - ETA: 1s - loss: 30.9816 - mean_absolute_error: 0.0540
Epoch 00001: val_loss improved from inf to 23.27128, saving model to weights/cfm-example-model-2018-07-05-best-fold-5-it-1.h5
120/120 [==============================] - 263s 2s/step - loss: 30.9233 - mean_absolute_error: 0.0540 - val_loss: 23.2713 - val_mean_absolute_error: 0.0465
Epoch 2/50
119/120 [============================>.] - ETA: 1s - loss: 22.2826 - mean_absolute_error: 0.0423
Epoch 00002: val_loss improved from 23.27128 to 21.84358, saving model to weights/cfm-example-model-2018-07-05-best-fold-5-it-1.h5
120/120 [==============================] - 254s 2s/step - loss: 22.2774 - mean_absolute_error: 0.0422 - val_loss: 21.8436 - val_mean_absolute_error: 0.0419
Epoch 3/50
119/120 [============================>.] - ETA: 1s - loss: 22.1326 - mean_absolute_error: 0.04

Epoch 3/50
119/120 [============================>.] - ETA: 1s - loss: 20.9196 - mean_absolute_error: 0.0394
Epoch 00003: val_loss did not improve
120/120 [==============================] - 253s 2s/step - loss: 20.9155 - mean_absolute_error: 0.0394 - val_loss: 21.4957 - val_mean_absolute_error: 0.0406
Epoch 4/50
119/120 [============================>.] - ETA: 1s - loss: 20.9616 - mean_absolute_error: 0.0394
Epoch 00004: val_loss did not improve
120/120 [==============================] - 227s 2s/step - loss: 20.9607 - mean_absolute_error: 0.0394 - val_loss: 21.4812 - val_mean_absolute_error: 0.0407
Epoch 00004: early stopping
Checkpoint. Fold 5 | Iteration 4 (LR 0.000004) -- Score --> 0.04064205 | 21.46868292 
Score Better than before (21.46868292 | 21.47333851)
Reducing LR to 0.000001
Stop. Too Much Iterations.
End of Training for Fold 5. Predicting.
End of Fold 5 | Score --> 21.4687 | 0.0406


In [20]:
cvscores

[(0.04151054384807746, 21.57486012776693),
 (0.03769589588046074, 21.247226905822753),
 (0.03940941691398621, 20.93321024576823),
 (0.042886945853630704, 21.558534240722658),
 (0.04064204879105091, 21.468682924906414)]

In [21]:
mean_cv_score = np.mean(cvscores, axis=0)
mean_cv_score

array([ 0.04042897, 21.35650289])

In [22]:
avg_test_score = np.mean(test_preds, axis=0)
out_name = 'results/%s-%d-Kfolds_%0.6f_%0.6f.csv' % (model_prefix, total_folds, mean_cv_score[0], mean_cv_score[1])
submission = pd.DataFrame({'ID':test_df["ID"], 'TARGET':avg_test_score.ravel()})
submission.to_csv(out_name , index=False)